# selenium


這段程式碼是一個爬取台灣手語線上辭典資料的工具，使用 Selenium 自動化瀏覽器操作，並將結果儲存到 CSV 檔案中，同時下載相關影片。
在使用這段程式碼之前，請確保已安裝以下必要的工具和套件：

套件安裝影片：[https://youtu.be/ximjGyZ93YQ?si=uDI2hXFOWxOKTHaF](https://youtu.be/ximjGyZ93YQ?si=uDI2hXFOWxOKTHaF)。
1. 安裝 Python (建議使用 Python 3.8 或以上版本，我用的是3.12.4)。
2. 安裝 Google Chrome 瀏覽器，並下載對應版本的 ChromeDriver。
3. 安裝必要的 Python 套件：Selenium、Requests 和 CSV。
4. 確保有權限在執行程式的目錄中建立資料夾和檔案，因為程式會下載影片並儲存到指定的目錄中。  
   - 預設影片儲存的資料夾名稱為：`downloads_multi`。  
   - 預設的csv 儲存檔案為: `sign_language_data_multi.csv`
5. 如果需要從 CSV 檔案中讀取關鍵字，請準備好格式正確的 CSV 檔案，並確保檔案中有名為 'input' 的欄位。  
   - 預設的input CSV 檔案名稱為學姊的csv檔案 : `datasets.csv`

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import requests
import os
import csv

def crawl_tsl(search_word, csv_filename="sign_language_data.csv", download_directory="downloads"):
    """
    爬取台灣手語線上辭典中特定關鍵字的資料，將資料追加到CSV檔案，並下載相關影片
    
    參數:
    search_word (str): 要搜尋的關鍵字
    csv_filename (str): CSV檔案名稱，預設為 "sign_language_data.csv"
    download_directory (str): 影片下載目錄，預設為 "downloads"
    
    回傳:
    int: 爬取到的資料筆數
    """
    # 確保下載目錄存在
    if not os.path.exists(download_directory):
        os.makedirs(download_directory)
        print(f"已創建下載目錄: {download_directory}")
    
    # 初始化 Chrome 瀏覽器
    driver = webdriver.Chrome()
    url = 'https://twtsl.ccu.edu.tw/TSL/'
    driver.get(url)
    
    try:
        # 搜尋關鍵字
        search_bar = driver.find_element(By.ID, "searchBar")
        search_bar.send_keys(search_word)
        
        # 等待並點擊搜尋按鈕
        wait = WebDriverWait(driver, 10)
        search_button = wait.until(EC.element_to_be_clickable((By.ID, "send")))
        driver.execute_script("arguments[0].scrollIntoView();", search_button)
        driver.execute_script("arguments[0].click();", search_button)
        
        # 等待結果載入
        time.sleep(3)
        
        # 擷取所有 .col-md-5 (每個影片的資訊)
        video_sections = driver.find_elements(By.CLASS_NAME, "col-md-5")
        
        if not video_sections:
            print(f"未找到 '{search_word}' 的相關資料")
            driver.quit()
            return 0
        
        data = []  # 存儲所有影片對應的資料
        
        # if video_sections:
        #     first_section = video_sections[0]
        #     html_content = first_section.get_attribute('outerHTML')
        #     print(f"第一個 video_section 的 HTML 結構:\n{html_content}")
        # else:
        #     print("video_sections 是空的，沒有內容物可以印出。")

        for section in video_sections:
            # **擷取 Words (所有 col-9 合併，用 "/" 分隔)**
            word_elements = section.find_elements(By.CLASS_NAME, "col-9")
            if word_elements:
                words = [el.text.strip() for el in word_elements if el.text.strip()] # 過濾掉空字串
                word = "/".join(words) # 如果有多個詞彙，則用 "/" 分隔
            else:
                word = "NaN"

            
            # 擷取 Sentence (sentenceHead)
            sentence_element = section.find_elements(By.CLASS_NAME, "sentenceHead")
            sentence = sentence_element[0].text.strip() if sentence_element else "NaN"
            
            # 擷取 Translation (text-left)
            translation_element = section.find_elements(By.CLASS_NAME, "text-left")
            translation = translation_element[0].text.strip() if translation_element else "NaN"
            
            # 擷取 Video 檔名
            video_element = section.find_elements(By.TAG_NAME, "video")
            video_filename = "NaN"
            video_url = None
            if video_element:
                source_element = video_element[0].find_elements(By.TAG_NAME, "source")
                if source_element:
                    video_url = source_element[0].get_attribute("src")
                    video_filename = os.path.basename(video_url)
            
            # 加入到結果
            data.append({
                "word": word, 
                "sentence": sentence, 
                "translation": translation, 
                "video": video_filename, 
                "video_url": video_url
            })
        
        # 爬取影片連結並下載影片
        video_elements = wait.until(EC.presence_of_all_elements_located((By.TAG_NAME, "video")))
        
        for idx, video_element in enumerate(video_elements):
            source_element = video_element.find_element(By.TAG_NAME, "source")
            video_url = source_element.get_attribute("src")
            print(f"影片網址: {video_url}")
            
            # 下載影片
            response = requests.get(video_url, stream=True)
            if response.status_code == 200:
                # 從URL獲取檔案名稱
                filename = os.path.basename(video_url)
                file_path = os.path.join(download_directory, filename)
                
                # 下載影片
                with open(file_path, 'wb') as f:
                    for chunk in response.iter_content(chunk_size=1024 * 1024):  # 1MB
                        if chunk:
                            f.write(chunk)
                print(f"影片已成功下載: {file_path}")
                
                # 更新對應的資料行
                if idx < len(data):  # 確保不超出 data 範圍
                    data[idx]["video"] = filename
            else:
                print(f"下載失敗，狀態碼: {response.status_code}")
        
        # 檢查CSV檔案是否存在，決定是否需要寫入標題
        file_exists = os.path.isfile(csv_filename)
        
        # 追加寫入CSV檔案
        with open(csv_filename, mode="a", newline="", encoding="utf-8") as file:
            writer = csv.writer(file)
            # 如果檔案不存在，寫入標題
            if not file_exists:
                writer.writerow(["Words", "Sentence", "Translation", "Video Filename"])
            
            # 寫入資料
            for row in data:
                writer.writerow([row["word"], row["sentence"], row["translation"], row["video"]])
        
        print(f"成功爬取 '{search_word}' 的 {len(data)} 筆資料並追加至 {csv_filename}")
        return len(data)
        
    except Exception as e:
        print(f"發生錯誤: {e}")
        return 0
    
    finally:
        # 關閉瀏覽器
        driver.quit()



In [ ]:
def read_keywords_from_csv(dataset_filename="datasets.csv"):
    """
    從CSV檔案中讀取input作為搜尋關鍵字
    
    參數:
    dataset_filename (str): 資料集CSV檔案名稱，預設為 "datasets.csv"
    
    回傳:
    list: 搜尋關鍵字列表
    """
    keywords = []
    
    try:
        with open(dataset_filename, mode='r', encoding='utf-8') as file:
            # 檢查CSV格式並讀取
            sample = file.read(1024)
            file.seek(0)  # 重置檔案指標
            
            # 嘗試判斷分隔符號
            dialect = csv.Sniffer().sniff(sample)
            reader = csv.reader(file, dialect)
            
            # 跳過標題行
            headers = next(reader)
            
            # 確認input欄位位置
            input_index = headers.index("input") if "input" in headers else 0
            
            # 讀取所有關鍵字
            for row in reader:
                if row and len(row) > input_index:
                    keyword = row[input_index].strip()
                    if keyword and keyword not in keywords:  # 避免重複關鍵字
                        keywords.append(keyword)
        
        print(f"從 {dataset_filename} 中讀取到 {len(keywords)} 個唯一關鍵字")
        return keywords
        
    except Exception as e:
        print(f"讀取CSV檔案時發生錯誤: {e}")
        return []

def process_all_keywords(dataset_filename="datasets.csv", result_csv="sign_language_data_multi.csv", download_dir="downloads_multi"):
    """
    處理所有從CSV檔案中讀取的關鍵字，爬取台灣手語資料
    
    參數:
    dataset_filename (str): 資料集CSV檔案名稱，預設為 "datasets.csv"
    result_csv (str): 結果CSV檔案名稱，預設為 "sign_language_data.csv"
    download_dir (str): 影片下載目錄，預設為 "downloads"
    """
    # 讀取關鍵字
    keywords = read_keywords_from_csv(dataset_filename)
    
    if not keywords:
        print("沒有找到可用的關鍵字，程式結束")
        return
    
    # 處理每個關鍵字
    total_entries = 0
    for i, keyword in enumerate(keywords):
        print(f"\n[{i+1}/{len(keywords)}] 正在處理關鍵字: {keyword}")
        entries = crawl_tsl(keyword, csv_filename=result_csv, download_directory=download_dir)
        total_entries += entries
    
    print(f"\n完成所有關鍵字處理")
    print(f"總共處理了 {len(keywords)} 個關鍵字")
    print(f"總共爬取了 {total_entries} 筆資料")
    print(f"資料已儲存至 {result_csv}")
    print(f"影片已下載至 {download_dir} 目錄")



In [ ]:
if __name__ == "__main__":
    # 從dataset.csv中讀取關鍵字並爬取台灣手語資料
    process_all_keywords()
    
    # 如果你想要自定義參數，可以這樣使用:
    # process_all_keywords(dataset_filename="my_dataset.csv", 
    #                      result_csv="my_results.csv", 
    #                      download_dir="my_videos")